In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import cv2
from sklearn.preprocessing import LabelEncoder



In [2]:
# Parameters
input_size = (224, 224)
batch_size = 42
num_classes = 6  # Number of sentiment classes
epochs = 50  # Adjust as needed



In [3]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)



In [4]:
# Define a function to preprocess the dataset
def preprocess_dataset(dataset_dir):
    data = []
    labels = []
    for emotion_label in os.listdir(dataset_dir):
        emotion_dir = os.path.join(dataset_dir, emotion_label)
        for image_file in os.listdir(emotion_dir):
            image_path = os.path.join(emotion_dir, image_file)
            image = load_img(image_path, target_size=input_size)
            image = img_to_array(image)
            
            # Check if it's a color image (3 channels) or grayscale
            if image.shape[-1] == 3:
                data.append(image)
            else:
                # Apply histogram equalization to grayscale images
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.equalizeHist(image)
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
                data.append(image)

            labels.append(emotion_label)
    
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    return np.array(data), labels


In [5]:
# Preprocess the training and testing datasets
train_dir = 'D:/Study/Digital Image Processing/Dataset/EmotionROI/training_testing_split/training'
test_dir = 'D:/Study/Digital Image Processing/Dataset/EmotionROI/training_testing_split/testing'
X_train, y_train = preprocess_dataset(train_dir)
X_test, y_test = preprocess_dataset(test_dir)



In [6]:
print(y_train)

[0 0 0 ... 5 5 5]


In [7]:
import joblib
from sklearn.preprocessing import LabelEncoder
# Convert one-hot encoded labels to 1D labels
y_train_labels = [np.argmax(label) for label in y_train]

# Fit the LabelEncoder to the training labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_labels)

# Save the fitted LabelEncoder
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [8]:
# One-hot encode the labels
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)



In [11]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout
#from tensorflow.keras.constraints import max_norm
# Load pre-trained models
def create_transfer_model(base_model, dropout_rate=0.5,l2_penalty=0.01):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(l2_penalty))(x) #batch normalization
    #Dense(1024, activation='relu', kernel_regularizer=l2(l2_penalty), kernel_constraint=max_norm(3))(x)
    x = Dropout(dropout_rate)(x)  # Add dropout layer
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

early_stopping = EarlyStopping(monitor='val_loss', patience=5)#early_stopping with patience 5

vgg_base = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=input_size + (3,))
resnet_base = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_size + (3,))
densenet_base = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False, input_shape=input_size + (3,))


vgg_model = create_transfer_model(vgg_base,l2_penalty=0.01)
resnet_model = create_transfer_model(resnet_base,l2_penalty=0.01)
densenet_model = create_transfer_model(densenet_base,l2_penalty=0.01)



In [12]:
# Compile the models
def compile_model(model):
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

compile_model(vgg_model)
compile_model(resnet_model)
compile_model(densenet_model)



In [13]:
# Data generators for training and testing
train_datagen = datagen.flow(X_train, y_train, batch_size=batch_size)
test_datagen = datagen.flow(X_test, y_test, batch_size=batch_size)



In [ ]:
# Train the models
vgg_history = vgg_model.fit(train_datagen, epochs=epochs, validation_data=test_datagen,callbacks=[early_stopping])
resnet_history = resnet_model.fit(train_datagen,epochs=epochs, validation_data=test_datagen,callbacks=[early_stopping])
densenet_history = densenet_model.fit(train_datagen, epochs=epochs, validation_data=test_datagen,callbacks=[early_stopping])



Epoch 1/50
33/33 [==============================] - 746s 22s/step - loss: 8.6314 - accuracy: 0.1681 - val_loss: 8.2655 - val_accuracy: 0.1633
Epoch 2/50
33/33 [==============================] - 736s 22s/step - loss: 8.0551 - accuracy: 0.1551 - val_loss: 7.8256 - val_accuracy: 0.2172
Epoch 3/50
33/33 [==============================] - 738s 22s/step - loss: 7.6070 - accuracy: 0.1789 - val_loss: 7.3777 - val_accuracy: 0.2071
Epoch 4/50
33/33 [==============================] - 737s 22s/step - loss: 7.1598 - accuracy: 0.1941 - val_loss: 6.9388 - val_accuracy: 0.2205
Epoch 5/50
33/33 [==============================] - 737s 22s/step - loss: 6.7385 - accuracy: 0.1861 - val_loss: 6.5319 - val_accuracy: 0.2003
Epoch 6/50
33/33 [==============================] - 743s 23s/step - loss: 6.3388 - accuracy: 0.2078 - val_loss: 6.1184 - val_accuracy: 0.2071
Epoch 7/50
33/33 [==============================] - 739s 22s/step - loss: 5.9562 - accuracy: 0.2287 - val_loss: 5.7788 - val_accuracy: 0.2323
Epoch 

In [ ]:
vgg_train_loss = vgg_history.history['loss']
vgg_val_loss = vgg_history.history['val_loss']
vgg_train_acc = vgg_history.history['accuracy']
vgg_val_acc = vgg_history.history['val_accuracy']

resnet_train_loss = resnet_history.history['loss']
resnet_val_loss = resnet_history.history['val_loss']
resnet_train_acc = resnet_history.history['accuracy']
resnet_val_acc = resnet_history.history['val_accuracy']

densenet_train_loss = densenet_history.history['loss']
densenet_val_loss = densenet_history.history['val_loss']
densenet_train_acc = densenet_history.history['accuracy']
densenet_val_acc = densenet_history.history['val_accuracy']


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(vgg_train_loss, label='VGG19 Training Loss')
plt.plot(vgg_val_loss, label='VGG19 Validation Loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('VGG19 Loss Over Time')

plt.subplot(1, 2, 2)
plt.plot(vgg_train_acc, label='VGG19 Training Accuracy')
plt.plot(vgg_val_acc, label='VGG19 Validation Accuracy')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('VGG19 Accuracy Over Time')

plt.show()


In [ ]:
# Save the VGG19 model
vgg_model.save('vgg_emotion_model.h5')

# Save the ResNet-50 model
resnet_model.save('resnet_emotion_model.h5')

# Save the DenseNet model
densenet_model.save('densenet_emotion_model.h5')


In [ ]:
# Calculate and print accuracy for each model
vgg_accuracy = vgg_history.history['accuracy'][-1]
resnet_accuracy = resnet_history.history['accuracy'][-1]
densenet_accuracy = densenet_history.history['accuracy'][-1]

print(f'VGG19 Accuracy: {vgg_accuracy * 100:.2f}%')
print(f'ResNet-50 Accuracy: {resnet_accuracy * 100:.2f}%')
print(f'DenseNet Accuracy: {densenet_accuracy * 100:.2f}%')


In [ ]:
# Load the saved models
from tensorflow.keras.models import load_model

# Load the VGG19 model
vgg_model = load_model('vgg_emotion_model.h5')

# Load the ResNet-50 model
resnet_model = load_model('resnet_emotion_model.h5')

# Load the DenseNet model
densenet_model = load_model('densenet_emotion_model.h5')

# Evaluate the models on the test data
vgg_evaluation = vgg_model.evaluate(X_test, y_test)
resnet_evaluation = resnet_model.evaluate(X_test, y_test)
densenet_evaluation = densenet_model.evaluate(X_test, y_test)

# Print the evaluation results
print("VGG19 Evaluation:")
print("Test Loss:", vgg_evaluation[0])
print("Test Accuracy:", vgg_evaluation[1])

print("\nResNet-50 Evaluation:")
print("Test Loss:", resnet_evaluation[0])
print("Test Accuracy:", resnet_evaluation[1])

print("\nDenseNet Evaluation:")
print("Test Loss:", densenet_evaluation[0])
print("Test Accuracy:", densenet_evaluation[1])


In [ ]:
encoded_classes = encoder.classes_
print(encoded_classes)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np


In [ ]:
k = 3  # You can adjust the number of folds as needed
vgg_accuracies = []
resnet_accuracies = []
densenet_accuracies = []


In [ ]:
kf = KFold(n_splits=k, shuffle=True, random_state=42)


In [ ]:
for train_index, test_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[test_index]
    
    vgg_model = create_transfer_model(vgg_base)
    resnet_model = create_transfer_model(resnet_base)
    densenet_model = create_transfer_model(densenet_base)
    
    compile_model(vgg_model)
    compile_model(resnet_model)
    compile_model(densenet_model)
    
    train_datagen = datagen.flow(X_train_fold, y_train_fold, batch_size=batch_size)
    
    vgg_history = vgg_model.fit(train_datagen, epochs=epochs, validation_data=(X_val_fold, y_val_fold),callbacks=[early_stopping])
    resnet_history = resnet_model.fit(train_datagen, epochs=epochs, validation_data=(X_val_fold, y_val_fold),callbacks=[early_stopping])
    densenet_history = densenet_model.fit(train_datagen, epochs=epochs, validation_data=(X_val_fold, y_val_fold),callbacks=[early_stopping])
    
    vgg_accuracies.append(vgg_history.history['val_accuracy'][-1])
    resnet_accuracies.append(resnet_history.history['val_accuracy'][-1])
    densenet_accuracies.append(densenet_history.history['val_accuracy'][-1])


In [ ]:
print(f"VGG19 Cross-Validation Mean Accuracy: {np.mean(vgg_accuracies) * 100:.2f}%")
print(f"VGG19 Cross-Validation Accuracy Std. Deviation: {np.std(vgg_accuracies) * 100:.2f}%")

print(f"ResNet-50 Cross-Validation Mean Accuracy: {np.mean(resnet_accuracies) * 100:.2f}%")
print(f"ResNet-50 Cross-Validation Accuracy Std. Deviation: {np.std(resnet_accuracies) * 100:.2f}%")

print(f"DenseNet Cross-Validation Mean Accuracy: {np.mean(densenet_accuracies) * 100:.2f}%")
print(f"DenseNet Cross-Validation Accuracy Std. Deviation: {np.std(densenet_accuracies) * 100:.2f}%")
